In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pdfminer
import json
import os
import sys
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LAParams, LTTextLine
import re

In [30]:
pdf_file_path = "/Users/pauldelseith/Documents/microsoft-hackathon-2023/Data/Modulhandbuecher/16_167_2021_Modulehandbook_Master's Program Finance and Information Management__20221118.pdf"

In [23]:
for page_layout in pdfminer.high_level.extract_pages("/Users/pauldelseith/Documents/microsoft-hackathon-2023/Data/Modulhandbuecher/16_167_2021_Modulehandbook_Master's Program Finance and Information Management__20221118.pdf"):
    for element in page_layout:
        print(element)

<LTTextBoxHorizontal(0) 28.346,782.912,157.856,802.712 'TUM School of Management \nTechnische Universität München\n'>
<LTTextBoxHorizontal(1) 28.346,686.647,196.370,710.647 'Module Catalog\n'>
<LTTextBoxHorizontal(2) 28.346,594.627,431.222,660.627 'M.Sc. Finance and Information Management (FIM)\nTUM School of Management\nTechnische Universität München\n'>
<LTTextBoxHorizontal(3) 28.346,523.446,109.592,549.446 'www.tum.de/\nwww.mgt.tum.de\n'>
<LTCurve 484.780,756.894,538.590,785.240>
<LTTextLineHorizontal 28.346,804.512,30.839,813.512 ' \n'>
<LTTextBoxHorizontal(0) 56.692,734.903,442.420,748.903 'Module Catalog: General Information and Notes to the Reader\n'>
<LTTextBoxHorizontal(1) 56.692,572.713,532.662,718.141 'What is the module catalog?\nOne of the central components of the Bologna Process consists in the modularization of university\ncurricula, that is, the transition of universities away from earlier seminar/lecture systems to\na modular system in which thematically-related cours

In [24]:
def extract_module_descriptions(pdf_path):
    text = pdfminer.high_level.extract_text(pdf_path)

    module_descriptions = []
    current_module_description = ""
    is_module_description = False

    for line in text.split('\n'):
        if "Module Description" in line:
            is_module_description = True
        elif line.strip() == "":
            is_module_description = False

        if is_module_description:
            current_module_description += line + '\n'
        elif current_module_description:
            module_descriptions.append(current_module_description)
            current_module_description = ""

    return module_descriptions

In [25]:
module_descriptions = extract_module_descriptions("/Users/pauldelseith/Documents/microsoft-hackathon-2023/Data/Modulhandbuecher/16_167_2021_Modulehandbook_Master's Program Finance and Information Management__20221118.pdf")

In [29]:
module_descriptions[3]

'Module Description\n'

In [31]:
pdf_text = pdfminer.high_level.extract_text(pdf_file_path)

In [32]:
pdf_text_split = pdf_text.split("Module Description")

In [33]:
len(pdf_text_split)

27

In [34]:
pdf_text_split[0]

'TUM School of Management \nTechnische Universität München\n\nModule Catalog\n\nM.Sc. Finance and Information Management (FIM)\nTUM School of Management\nTechnische Universität München\n\nwww.tum.de/\nwww.mgt.tum.de\n\n \n\x0cModule Catalog: General Information and Notes to the Reader\n\nWhat is the module catalog?\nOne of the central components of the Bologna Process consists in the modularization of university\ncurricula, that is, the transition of universities away from earlier seminar/lecture systems to\na modular system in which thematically-related courses are bundled together into blocks, or\nmodules. \nThis module catalog contains descriptions of all modules offered in the course of study.\nServing the goal of transparency in higher education, it provides students, potential students and\nother internal and external parties with information on the content of individual modules, the goals\nof academic qualification targeted in each module, as well as their qualitative and quanti

In [35]:
pdf_text_split[1]

'\n\nWI001271: Entrepreneurship | Entrepreneurship\n\nVersion of module description: Gültig ab winterterm 2020/21\n\nModule Level:\nMaster\n\nLanguage:\nEnglish\n\nDuration:\none semester\n\nFrequency:\nwinter/summer\nsemester\n\nCredits:*\n6\n\nTotal Hours:\n180\n\nSelf-study Hours:\n120\n\nContact Hours:\n60\n\nNumber of credits may vary according to degree program. Please see Transcript of Records.\n\nDescription of Examination Method:\nThe examination consists of several exam courses (Prüfungsparcour). A semester-long project\nwork, which ends in the delivery of a business plan and in a presentation. The presentation\nincludes a prototype-demo of the developed product or service and a reflection on how innovative\nentrepreneurs from the guest lectures lead high-tech organizations. Through the projectwork, it is\nassessed how well the participants can identify and implement business opportunities. In teams\nstudents recognize the needs and demands of the customers. Through customer 

In [36]:
pdf_text_split[2]

'\n\nWI001287: Basics of FIM | Basics of FIM\n\nVersion of module description: Gültig ab winterterm 2020/21\n\nModule Level:\nMaster\n\nLanguage:\nEnglish\n\nDuration:\none semester\n\nFrequency:\nwinter semester\n\nCredits:*\n6\n\nTotal Hours:\n180\n\nSelf-study Hours:\n120\n\nContact Hours:\n60\n\nNumber of credits may vary according to degree program. Please see Transcript of Records.\n\nDescription of Examination Method:\nThe module examination is based on a written exam (60 min). Students have to show their\nunderstanding of the core concepts being presented within the different courses. Students have to\nprove their understanding of the quantitative methods being part of the lectures. The exam might\nalso include calculations.  \n\nRepeat Examination:\nEnd of Semester\n\n(Recommended) Prerequisites:\n\nContent:\nBasics of Finance, financial management as key function of corporations, core methods of value-\nand risk management, impact of long- and short-term orientation in corpor

In [56]:
pattern = r'\n\n(.*?)\n\n'
match = re.search(pattern, pdf_text_split[3], re.DOTALL)

if match:
    print(match.group(1))
else:
    print("Test")

WI000234: Value-based Management | Value-based Management


In [57]:
pdf_text_dict = {}
for text_chunk in pdf_text_split[1:-1]:
    match = re.search(pattern, text_chunk, re.DOTALL)
    if match:
        pdf_text_dict[match.group(1)] = text_chunk.replace(match.group(1), "")
    else:
        print("Test")

In [59]:
with open("module_descriptions_dict.json", 'w') as json_file:
    json.dump(pdf_text_dict, json_file)

In [60]:
list(pdf_text_dict.keys())

['WI001271: Entrepreneurship | Entrepreneurship',
 'WI001287: Basics of FIM | Basics of FIM',
 'WI000234: Value-based Management | Value-based Management',
 'IN2028: Business Analytics and Machine Learning | Business Analytics\nand Machine Learning',
 'MA9972: Discrete Time Finance (FIM) | Discrete Time Finance (FIM)',
 'MA9973: Continuous Time Finance (FIM) | Continuous Time Finance\n(FIM)',
 'WIBT0001: Business Process Management & Digital Innovation |\nBusiness Process Management & Digital Innovation ',
 'WIBT0002: Digital Energy Management | Digital Energy Management',
 'WIBT0003: Digital Disruption, Innovation and Transformation | Digital\nDisruption, Innovation and Transformation',
 'WI001267: Advanced Corporate Finance | Advanced Corporate Finance',
 'WI001268: Venture Capital | Venture Capital',
 'WI001269: International Accounting | International Accounting\nFor students in the FIM Master',
 'WI001270: Behavioral Finance | Behavioral Finance',
 'WI001272: Machine Learning | Ma

In [61]:
pdf_text_dict['WI000813: Technology Entrepreneurship Lab | Technology\nEntrepreneurship Lab']

'\n\n\n\nVersion of module description: Gültig ab summerterm 2018\n\nModule Level:\nMaster\n\nLanguage:\nEnglish\n\nDuration:\none semester\n\nFrequency:\nwinter/summer\nsemester\n\nCredits:*\n6\n\nTotal Hours:\n180\n\nSelf-study Hours:\n120\n\nContact Hours:\n60\n\nNumber of credits may vary according to degree program. Please see Transcript of Records.\n\nDescription of Examination Method:\nThe  grading is based on a project work.\nWith the project work students show their understanding of the processes associated with the\nrecognition and development of entrepreneurial opportunities. Students show that they are able\nto analyze the development of entrepreneurial teams. Moreover, they show their ability to apply\ncoaching tools.\nThroughout the project work each student has to hand in regular written documentation of\nmaximum one page in which to describe the continuous development of the entrepreneurial idea\nas well as the team (60%). At the end of the project work each student has

In [64]:
# generate a formatted .txt file from the pdf_text_dict contents
with open("module_descriptions.txt", 'w') as txt_file:
    key = 'WI000813: Technology Entrepreneurship Lab | Technology\nEntrepreneurship Lab'
    value = pdf_text_dict[key]
    txt_file.write(key + "\n")
    txt_file.write(value + "\n\n")

In [66]:
categories = ['Module Level:', 'Language:', 'Duration:', 
              'Frequency:', 'Credits:', 'Total Hours:', 
              'Self-study Hours:', 'Contact Hours:', 'Description of Examination Method:',
              'Repeat Examination:', '(Recommended) Prerequisites:', 'Content:', 'Intended Learning Outcomes:', 
              'Teaching and Learning Methods:', 'Media:', 'Reading List:', 'Responsible for Module:', 
              'Courses (Type of course, Weekly hours per semester), Instructor:']

In [75]:
key = 'WI001271: Entrepreneurship | Entrepreneurship'
value = pdf_text_dict[key]
value

'\n\n\n\nVersion of module description: Gültig ab winterterm 2020/21\n\nModule Level:\nMaster\n\nLanguage:\nEnglish\n\nDuration:\none semester\n\nFrequency:\nwinter/summer\nsemester\n\nCredits:*\n6\n\nTotal Hours:\n180\n\nSelf-study Hours:\n120\n\nContact Hours:\n60\n\nNumber of credits may vary according to degree program. Please see Transcript of Records.\n\nDescription of Examination Method:\nThe examination consists of several exam courses (Prüfungsparcour). A semester-long project\nwork, which ends in the delivery of a business plan and in a presentation. The presentation\nincludes a prototype-demo of the developed product or service and a reflection on how innovative\nentrepreneurs from the guest lectures lead high-tech organizations. Through the projectwork, it is\nassessed how well the participants can identify and implement business opportunities. In teams\nstudents recognize the needs and demands of the customers. Through customer feedback, field\ninterviews and contextual ob

In [73]:
for category in categories:
    key = 'WI001271: Entrepreneurship | Entrepreneurship'
    value = pdf_text_dict[key]
    pattern = category + r'\n(.*?)\n'
    match = re.search(pattern, value, re.DOTALL)
    if match:
        print(category)
        print(match.group(1))
    else:
        print(f"--{category} not found--")

Module Level:
Master
Language:
English
Duration:
one semester
Frequency:
winter/summer
--Credits: not found--
Total Hours:
180
Self-study Hours:
120
Contact Hours:
60
Description of Examination Method:
The examination consists of several exam courses (Prüfungsparcour). A semester-long project
Repeat Examination:
Next semester
--(Recommended) Prerequisites: not found--
Content:
In a creative atmosphere, the participants learn to think through and present a business idea in the
Intended Learning Outcomes:
At the end of the seminar the students will be able to:
Teaching and Learning Methods:
The module is held in seminar-style: The lecturers are entrepreneurs, serial founders, coaches,
Media:
- Videos
Reading List:
- Timmons, Jeffry A. / Spinelli, Stephen (2009): New Venture Creation,7thedition, McGraw Hill
Responsible for Module:
Bücken, Oliver; Dipl.-Kfm. (Univ.)
--Courses (Type of course, Weekly hours per semester), Instructor: not found--


In [85]:
categorized = {}
key = 'WI001287: Basics of FIM | Basics of FIM'
value = pdf_text_dict[key]
chopped_rest = value
temp = chopped_rest.split(category)
chopped_rest = temp[1]
for category in categories:
    temp = chopped_rest.split(category)
    categorized[category] = temp[0]
    if len(temp) > 1:
        chopped_rest = temp[1]

In [86]:
# print every category and its value nicely formatted
for category in categorized.keys():
    print(category)
    print(categorized[category])
    print("\n")

Module Level:
 
Investment & Financing (FIM) (WI001287) (Vorlesung, 2 SWS)
Nardini M, Bayer C, Braun R, Egger A, van Dun C, Wagon F

Business & Information Systems Engineering (FIM) (WI001287) (Vorlesung, 2 SWS)
Röglinger M, van Dun C, Bayer C, Egger A, Wagon F

Stochastic Processes (FIM) (WI001287) (Vorlesung, 2 SWS)
Zagst R, Rauscher M, Bayer C, Egger A, van Dun C, Wagon F
For further information in this module, please click campus.tum.de or here.

Module Catalog of the study program M.Sc. Finance and Information Management (FIM)
Generated on 18.11.2022

11 of 68

Core area | Kernbereich

Core area | Kernbereich

At least 48 credits must be selected from the core area within the framework of the elective
modules. These examination credits can be partially completed at the University of Bayreuth.
These modules are accordingly assigned a module number beginning with the abbreviation
WIBT. The applicable elective module catalog will be announced by the Faculty of Business and
Economics

In [84]:
print(pdf_text_dict['WI001287: Basics of FIM | Basics of FIM'])





Version of module description: Gültig ab winterterm 2020/21

Module Level:
Master

Language:
English

Duration:
one semester

Frequency:
winter semester

Credits:*
6

Total Hours:
180

Self-study Hours:
120

Contact Hours:
60

Number of credits may vary according to degree program. Please see Transcript of Records.

Description of Examination Method:
The module examination is based on a written exam (60 min). Students have to show their
understanding of the core concepts being presented within the different courses. Students have to
prove their understanding of the quantitative methods being part of the lectures. The exam might
also include calculations.  

Repeat Examination:
End of Semester

(Recommended) Prerequisites:

Content:
Basics of Finance, financial management as key function of corporations, core methods of value-
and risk management, impact of long- and short-term orientation in corporate management, impact
of financial decisions on customers, employees and the society

In [8]:
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from bs4 import BeautifulSoup

In [5]:
pdf_path = pdf_path = "/Users/pauldelseith/Documents/microsoft-hackathon-2023/Data/Modulhandbuecher/16_167_2021_Modulehandbook_Master's Program Finance and Information Management__20221118.pdf"
loader = PDFMinerPDFasHTMLLoader(pdf_path)

In [6]:
data = loader.load()[0]   # entire PDF is loaded as a single Document

In [9]:
soup = BeautifulSoup(data.page_content,'html.parser')
content = soup.find_all('div')

In [11]:
content
with open("soup.txt", 'w') as html_file:
    html_file.write(str(content))

In [30]:
module_descriptions = {}
current_module_name = ""
for idx, element in enumerate(content):
    if 'Module Description' in str(element):
        module_name = content[idx+1].text
        module_name = module_name.replace('\n', '')
        module_descriptions[module_name] = []
        current_module_name = module_name
    elif current_module_name != "" and element.text.replace('\n', '') != current_module_name:
        module_descriptions[current_module_name].append(element)

In [31]:
list(module_descriptions.keys())

['WI001271: Entrepreneurship | Entrepreneurship',
 'WI001287: Basics of FIM | Basics of FIM',
 'WI000234: Value-based Management | Value-based Management',
 'IN2028: Business Analytics and Machine Learning | Business Analyticsand Machine Learning',
 'MA9972: Discrete Time Finance (FIM) | Discrete Time Finance (FIM)',
 'MA9973: Continuous Time Finance (FIM) | Continuous Time Finance(FIM)',
 'WIBT0001: Business Process Management & Digital Innovation |Business Process Management & Digital Innovation ',
 'WIBT0002: Digital Energy Management | Digital Energy Management',
 'WIBT0003: Digital Disruption, Innovation and Transformation | DigitalDisruption, Innovation and Transformation',
 'WI001267: Advanced Corporate Finance | Advanced Corporate Finance',
 'WI001268: Venture Capital | Venture Capital',
 'WI001269: International Accounting | International AccountingFor students in the FIM Master',
 'WI001270: Behavioral Finance | Behavioral Finance',
 'WI001272: Machine Learning | Machine Lear

In [32]:
module_descriptions['WI001271: Entrepreneurship | Entrepreneurship']

[<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:56px; top:4655px; width:291px; height:11px;"><span style="font-family: ArialMT; font-size:11px">Version of module description: Gültig ab winterterm 2020/21
 <br/></span></div>,
 <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:68px; top:4694px; width:73px; height:25px;"><span style="font-family: Arial-BoldMT; font-size:11px">Module Level:
 <br/></span><span style="font-family: ArialMT; font-size:11px">Master
 <br/></span></div>,
 <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:188px; top:4694px; width:55px; height:25px;"><span style="font-family: Arial-BoldMT; font-size:11px">Language:
 <br/></span><span style="font-family: ArialMT; font-size:11px">English
 <br/></span></div>,
 <div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:309px; top:4694px; width:66px; height:25px;"><span style="font-family: Arial-

In [33]:
categories = ['Module Level:', 'Language:', 'Duration:', 
              'Frequency:', 'Credits:', 'Total Hours:', 
              'Self-study Hours:', 'Contact Hours:', 'Description of Examination Method:',
              'Repeat Examination:', '(Recommended) Prerequisites:', 'Content:', 'Intended Learning Outcomes:', 
              'Teaching and Learning Methods:', 'Media:', 'Reading List:', 'Responsible for Module:', 
              'Courses (Type of course, Weekly hours per semester), Instructor:']

In [59]:
key = 'WI001271: Entrepreneurship | Entrepreneurship'
value = module_descriptions[key]
module_descr_classified = {}
for key, value in module_descriptions.items():
    module_descr_classified[key] = {}
    for val in value:
        if any(categorie in val.text for categorie in categories):
            txt_split = val.text.split(":")  
            extr_categorie = txt_split[0]
            if extr_categorie == "Credits":
                extr_content = txt_split[1].replace("*","")
            else:
                extr_content = txt_split[1]
            module_descr_classified[key][extr_categorie] = extr_content.replace('\n', '')

In [60]:
len(list(module_descr_classified.keys()))

26

In [61]:
for key, _ in module_descr_classified.items():
    print(len(module_descr_classified['WI001271: Entrepreneurship | Entrepreneurship']))

18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18


In [62]:
with open("module_descriptions_parsed.json", 'w', encoding="utf-8") as json_file:
    json.dump(module_descr_classified, json_file, ensure_ascii=False)

In [57]:
with open("module_descriptions_parsed.json", 'r', encoding="utf-8") as json_file:
    test = json.load(json_file)